<a href="https://colab.research.google.com/github/foiegreis/Udacity-waymo-object-detection/blob/main/Explore_augmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore data augmentations

This notebook will be used to test different data augmentation strategies. In your config file, you can modify the augmentations by adding `data_augmentation_options`. For example, I would like to add a random RGB to gray transform with a probability of 0.2:
```
data_augmentation_options {
    random_rgb_to_gray {
    probability: 0.2
    }
```

A complete list of augmentation is provided the [`preprocessor.proto` file of the Tf Object Detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto).

### Prepare the environment

Install tensorflow object detection api dependencies

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/models/research/')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/models/research/slim/')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/models/')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/object_detection/')
libs_path = '/content/gdrive/MyDrive/GoogleColab/lib'
%cd $libs_path

/content/gdrive/MyDrive/GoogleColab/lib


In [3]:
%cd $libs_path
import os
if 'models' in os.listdir(libs_path):
  %rm -r models
!git clone --depth 1 https://github.com/tensorflow/models


/content/gdrive/MyDrive/GoogleColab/lib
Cloning into 'models'...
remote: Enumerating objects: 3339, done.
remote: Counting objects: 100% (3339/3339), done.
remote: Compressing objects: 100% (2772/2772), done.
remote: Total 3339 (delta 880), reused 1439 (delta 517), pack-reused 0
Receiving objects: 100% (3339/3339), 34.34 MiB | 10.37 MiB/s, done.
Resolving deltas: 100% (880/880), done.
Checking out files: 100% (3040/3040), done.


In [4]:
%cd $libs_path/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/gdrive/MyDrive/GoogleColab/lib/models/research
Processing /content/gdrive/MyDrive/GoogleColab/lib/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.2 MB 13.6 MB/s 
     |████████████████████████████████| 352 kB 57.3 MB/s 
     |████████████████████████████████| 2.2 MB 40.9 MB/s 
     |████████████████████████████████| 23.4 MB 1.5 MB/s 
     |████████████████████████████████| 99 kB 8.7 MB/s 
     |████████████████████████████████| 237 kB 54.4 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 636 kB 52.3 MB/s 
     |██████████████

In [5]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q pycocotools

Selecting previously unselected package python-bs4.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [6]:
!pip3 uninstall opencv-python-headless==4.5.5.62
!pip3 install --target=$libs_path opencv-python-headless==4.5.2.52
##NOTE: REQUIRES USER INPUT

Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.5.5.64.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-65fa80df.so.58.134.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-8ef5c7db.so.58.76.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-9c768859.so.56.70.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-09fe7800.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dis

In [7]:
import tensorflow as tf
print(tf.__version__)

2.8.0


# Colab Paths

![](https://drive.google.com/uc?id=1hzVreauXhuWiE_QNXL2sKd8Zb3p5pY8M)

In [8]:
path = '/content/gdrive/MyDrive/GoogleColab'
udacity_path = path + '/UDACITY'
sdc_path = udacity_path + '/SDC'
project_path = sdc_path + '/1-waymo-object-detection'

data_path = project_path + '/data'
waymo_path = data_path + '/waymo'
train_data_path = waymo_path + '/train'

folder_path = project_path + '/Udacity-waymo-object-detection'
workspace_path = folder_path + '/home/workspace'

### Helper functions

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

In [10]:
def recenter_image(image):
    # ssd preprocessing
    image += [123.68, 116.779, 103.939]
    return image

def display_instances(image, bboxes, classes):
    image = recenter_image(image)
    w, h, _ = image.shape
    # resize the bboxes
    bboxes[:, [0, 2]] *= w
    bboxes[:, [1, 3]] *= h
    
    f, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image.astype(np.uint8))
    for bb, cl in zip(bboxes, classes):
        y1, x1, y2, x2 = bb
        rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor='r', linewidth=2)
        ax.add_patch(rec)
    plt.show()

def display_batch(batch):
    # get images, bboxes and classes
    batched_images = batch[0]['image'].numpy()
    batched_bboxes = batch[1]['groundtruth_boxes'].numpy()
    batched_classes = batch[1]['groundtruth_classes'].numpy()
    num_bboxes = batch[1]['num_groundtruth_boxes'].numpy()
    batch_size = batched_images.shape[0]
    for idx in range(batch_size):
        display_instances(batched_images[idx, ...], 
                          batched_bboxes[idx, :num_bboxes[idx], :],
                          batched_classes[idx, ...])

### Augmentations

Below, the list of the used augmentation techniques:

- randiom horizontal flip [default]
- ssd random crop 
- random rgb to gray
- random adjust brightness [max delta: 0.3]
- random adjust contrast [min delta: 0.8 max delta: 1.25]
- random adjust hue [max delta: 0.02]
- random adjust saturation [min delta: 0.8, max_delta: 1.1]
- random image scale
- random black patches [max black patches: 4, probability: 0.3, size to image ratio: 0.07]


These techniques were added in order to manage the dataset limitations found during Exploratory Data Analysis (see colab notebook)

Assessing data imbalance remains a very difficult task, expecially working with APIs. In fact I tried to apply some custom approaches but wasn't so sure about the outcome. So, the data augmentation below will cover the other problems, leaving class imbalance to a loss approach via loss weights


### Display augmentations

In [21]:
!git pull

Already up to date.


In [12]:
workspace_path = workspace_path
%cd $workspace_path

/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/Udacity-waymo-object-detection/home/workspace


In [13]:
!ls

build		     Exploratory_Data_Analysis.ipynb  pipeline.config
CODEOWNERS	     filenames.txt		      __pycache__
create_splits.py     inference_video.py		      README.md
download_process.py  __init__.py		      utils_colab.py
edit_config.py	     label_map.pbtxt		      utils.py
experiments	     LICENSE.md


In [22]:
from utils import get_train_input

experiment1_config = workspace_path + '/experiments/experiment1/pipeline_new.config'
train_dataset = get_train_input(experiment1_config)



INFO:tensorflow:Reading unweighted datasets: ['/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/data/waymo/train/segment-10153695247769592104_787_000_807_000_with_camera_labels.tfrecord', '/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/data/waymo/train/segment-10023947602400723454_1120_000_1140_000_with_camera_labels.tfrecord', '/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/data/waymo/train/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord', '/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/data/waymo/train/segment-10072140764565668044_4060_000_4080_000_with_camera_labels.tfrecord', '/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/data/waymo/train/segment-10107710434105775874_760_000_780_000_with_camera_labels.tfrecord', '/content/gdrive/MyDrive/GoogleColab/UDACITY/SDC/1-waymo-object-detection/data/waymo/train/segment-10096619443888687526

In [23]:
%matplotlib inline

for batch in train_dataset.take(2):
    display_batch(batch)

Output hidden; open in https://colab.research.google.com to view.